In [2]:
## LIBRARIES
# ---

# Selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains

# Options driver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

# Dataframes
import pandas as pd
import itertools
import os

# Simulating human behavior
import time
from time import sleep
import random

# Clear data
import unidecode

# Json files
import json
import re
import numpy as np
import itertools
from pandas import json_normalize

# To use explicit waits
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Download files
import urllib.request
import requests
from openpyxl import Workbook

# pytesseract
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [42]:
options = Options()
service = Service( ChromeDriverManager().install( ) )
driver  = webdriver.Chrome( service = service )
driver.maximize_window()

url = f'https://enlinea.sunedu.gob.pe/'
driver.get( url )

wait = WebDriverWait( driver, 60 )

In [43]:
verificar_titulos_popup = driver.find_element( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' )
verificar_titulos_popup.click()

In [44]:
frame_popup = driver.find_element( By.XPATH, '//*[@id="ifrmShowFormConstancias"]' )
driver.switch_to.frame( frame_popup )

In [45]:
# dni = '32224811'
dni = '70313538'

In [46]:
dni_campo = driver.find_element( By.XPATH, '//*[@id="doc"]' )
dni_campo.clear()
dni_campo.send_keys( dni )

In [47]:
captcha       = driver.find_element( By.XPATH, '//*[@id="captchaImg"]/img' )
captcha_image = captcha.screenshot_as_png
with open( 'image.png', 'wb' ) as f:
    f.write( captcha_image )
    
clave = pytesseract.image_to_string( Image.open( 'image.png') ).strip()

In [48]:
campo_captcha = driver.find_element(By.XPATH, '//*[@id="captcha"]' )
campo_captcha.clear()
campo_captcha.send_keys( clave )

In [49]:
buscar_boton = driver.find_element( By.XPATH, '//*[@id="buscar"]' )
buscar_boton.click()

In [13]:
result_sections = driver.find_elements( By.XPATH, '//*[@id="finalData"]/tr' )
sections = {f'section {i}':result_section.text for i,result_section in enumerate(result_sections)}

In [14]:
sections = {}
for i, result_section in enumerate(result_sections):
    section_name = f'section {i}'
    sections[section_name] = result_section.text

In [15]:
sections

{'section 0': 'CALDAS CAÑARI, ROMEL DAVID\nDNI 32224811\nLICENCIADO EN CIENCIAS DE LA EDUCACION\n\nFecha de diploma:\nModalidad de estudios: -\nUNIVERSIDAD NACIONAL DE EDUCACIÓN ENRIQUE GUZMÁN Y VALLE\nPERU',
 'section 1': 'CALDAS CAÑARI, ROMEL DAVID\nDNI 32224811\nLICENCIADO EN EDUCACION. ESPECIALIDAD:\n\nFecha de diploma: 18/05/1998\nModalidad de estudios: -\nUNIVERSIDAD NACIONAL DE EDUCACIÓN ENRIQUE GUZMÁN Y VALLE\nPERU',
 'section 2': 'CALDAS CAÑARI, ROMEL DAVID\nDNI 32224811\nBACHILLER EN CIENCIAS DE LA EDUCACION\n\nFecha de diploma: 22/08/1997\nModalidad de estudios: -\n\nFecha matrícula: Sin información (***)\nFecha egreso: Sin información (***) UNIVERSIDAD NACIONAL DE EDUCACIÓN ENRIQUE GUZMÁN Y VALLE\nPERU',
 'section 3': 'CALDAS CAÑARI, ROMEL DAVID\nDNI 32224811\nMAESTRO EN EDUCACION CON MENCION EN DOCENCIA UNIVERSITARIA Y GESTION EDUCATIVA\n\nFecha de diploma: 12/10/17\nModalidad de estudios: PRESENCIAL\n\nFecha matrícula: 16/04/2016\nFecha egreso: 12/11/2017 UNIVERSIDAD SAN 

In [53]:
rows = []
sections = {}
result_sections = driver.find_elements( By.XPATH, '//*[@id="finalData"]/tr' )
for i, result in enumerate( result_sections ):
    section_name = f'section {i}'
    boxes = result.find_elements( By.XPATH, f'//*[@id="finalData"]/tr[{ i }]/td' )
    for box in boxes:
        text = box.text
        sections[section_name] = box.text
        
        row = {'title':'hola' }
        row.update(sections)
        rows.append(row)
        
df = pd.DataFrame(rows)
df

""


In [ ]:
//*[@id="finalData"]/tr

In [33]:
# SI FUNCIONAAA

rows = []
sections = {}
result_sections = driver.find_elements(By.XPATH, '//*[@id="finalData"]/tr')

for i, result in enumerate(result_sections):
    
    section_name = f'section {i}'
    boxes = result.find_elements(By.XPATH, f'//*[@id="finalData"]/tr[{i + 1}]/td')  # Ajuste del índice
    for j, box in enumerate(boxes):
        text = box.text
        sections[f'{section_name} {j + 1}'] = text  # Ajuste del índice

# Crear una fila por persona con todas las secciones
row = {'title': 'hola'}
row.update(sections)
rows.append(row)

df = pd.DataFrame(rows)

df

,title,section 0 1,section 0 2,section 0 3,section 0 4,section 1 1,section 1 2,section 1 3,section 1 4,section 2 1,section 2 2,section 2 3,section 2 4,section 3 1,section 3 2,section 3 3,section 3 4
0,hola,"CALDAS CAÑARI, ROMEL DAVID\nDNI 32224811",LICENCIADO EN CIENCIAS DE LA EDUCACION\n\nFech...,UNIVERSIDAD NACIONAL DE EDUCACIÓN ENRIQUE GUZM...,,"CALDAS CAÑARI, ROMEL DAVID\nDNI 32224811",LICENCIADO EN EDUCACION. ESPECIALIDAD:\n\nFech...,UNIVERSIDAD NACIONAL DE EDUCACIÓN ENRIQUE GUZM...,,"CALDAS CAÑARI, ROMEL DAVID\nDNI 32224811",BACHILLER EN CIENCIAS DE LA EDUCACION\n\nFecha...,UNIVERSIDAD NACIONAL DE EDUCACIÓN ENRIQUE GUZM...,,"CALDAS CAÑARI, ROMEL DAVID\nDNI 32224811",MAESTRO EN EDUCACION CON MENCION EN DOCENCIA U...,UNIVERSIDAD SAN PEDRO\nPERU,


In [26]:
df.to_excel( 'hola.xlsx' )

In [54]:
rows            = []
sections        = {}
result_sections = driver.find_elements( By.XPATH, '//*[@id="finalData"]/tr' )

for i, result in enumerate( result_sections ):
    
    boxes = result.find_elements( By.XPATH, f'//*[@id="finalData"]/tr[{ i + 1 }]/td' )
    
    for j, box in enumerate( boxes ):
        
        boxes_names_list = { 0 : 'datos', 1 : 'grado_fecha', 2 : 'institucion', 3 : 'otro' }
        box_name         = boxes_names_list.get( j )   
        text             = box.text
        sections[ f'{ box_name }_{ i }' ] = text

row = {'title': 'hola'}
row.update( sections )
rows.append( row )

df = pd.DataFrame( rows )

df

,title,datos_0,grado_fecha_0,institucion_0,otro_0
0,hola,"CALDAS VELASQUEZ, JOSUE DANIEL\nDNI 70313538",Bachiller en Ciencias Sociales con Mención en ...,PONTIFICIA UNIVERSIDAD CATÓLICA DEL PERÚ\nPERU,


In [27]:
df = pd.DataFrame({'texto': [text]})
df

,texto
0,Resultado\nGraduado Grado o Título Institución...


In [30]:
import pandas as pd
import re

patrones = [
    'Grado o Título Institución',
    'Fecha de diploma:',
    'Modalidad de estudios:',
    'Fecha matrícula:',
    'Fecha egreso:'
]

# Crear un diccionario para almacenar las listas de valores para cada elemento
elementos = {patron: [] for patron in patrones}

# Iterar sobre cada fila del DataFrame
for index, row in df.iterrows():
    texto = row['texto']
    
    # Iterar sobre cada patrón y buscar coincidencias en el texto
    for patron in patrones:
        matches = re.finditer(patron + r'[\n:]+(.+?)(?=\n|$)', texto)
        elementos[patron].extend([match.group(1).strip() for match in matches])

# Crear un nuevo DataFrame con las columnas dinámicas
df_resultado = pd.DataFrame(elementos)

# Renombrar las columnas para que sean únicas
df_resultado.columns = [f'{col}' for col in df_resultado.columns]

# Concatenar el nuevo DataFrame con el orig

ValueError: All arrays must be of the same length

In [31]:
elementos

{'Grado o Título Institución': ['CALDAS CAÑARI, ROMEL DAVID'],
 'Fecha de diploma:': ['Modalidad de estudios: -'],
 'Modalidad de estudios:': [],
 'Fecha matrícula:': [],
 'Fecha egreso:': []}

In [21]:
df.columns

Index(['Apellido', 'Nombre', 'DNI', 'Fecha Diploma', 'Grado 1',
       'Institución 1', 'Modalidad 1', 'Fecha Matrícula 1', 'Fecha Egreso 1'],
      dtype='object')

In [41]:
import pandas as pd
import re

# Supongamos que tu DataFrame se llama df y la columna de texto es 'texto_columna'
# df = ...

columns = ['Apellido', 'Nombre', 'DNI', 'Grado', 'Fecha Diploma', 'Modalidad', 'Fecha Matrícula', 'Fecha Egreso']

# Lista de patrones que queremos buscar en el texto
patron = r'(\w+), (\w+ [\w ]+)\nDNI (\d+)\n([^\n]+)\n\nFecha de diploma: (\d{1,2}/\d{1,2}/\d{4})\nModalidad de estudios: ([^\n]+)\n\nFecha matrícula: ([^\n]+)\nFecha egreso: ([^\n]+) ([^\n]+)\n'

# Crear un diccionario para almacenar las listas de valores para cada elemento
elementos = {col: [] for col in columns}

# Iterar sobre cada fila del DataFrame
for index, row in df.iterrows():
    texto = row['texto']
    
    # Buscar coincidencias en el texto
    match = re.search(patron, texto)
    if match:
        elementos['Apellido'].append(match.group(1))
        elementos['Nombre'].append(match.group(2))
        elementos['DNI'].append(match.group(3))
        elementos['Grado'].append(match.group(4))
        elementos['Fecha Diploma'].append(match.group(5))
        elementos['Modalidad'].append(match.group(6))
        elementos['Fecha Matrícula'].append(match.group(7))
        elementos['Fecha Egreso'].append(match.group(8) + " " + match.group(9))
        # elementos['Institución'].append(match.group(10))

# Crear un nuevo DataFrame con las columnas específicas
df_resultado = pd.DataFrame(elementos)

# Concatenar el nuevo DataFrame con el original
df = pd.concat([df, df_resultado], axis=1)

# Mostrar el resultado
df

,texto,Apellido,Nombre,DNI,Grado,Fecha Diploma,Modalidad,Fecha Matrícula,Fecha Egreso,Apellido,...,Fecha Matrícula,Fecha Egreso,Apellido,Nombre,DNI,Grado,Fecha Diploma,Modalidad,Fecha Matrícula,Fecha Egreso
0,Resultado\nGraduado Grado o Título Institución...,CAÑARI,ROMEL DAVID,32224811,BACHILLER EN CIENCIAS DE LA EDUCACION,22/08/1997,-,Sin información (***),Sin información (***) UNIVERSIDAD NACIONAL DE ...,CAÑARI,...,Sin información (***),Sin información (***) UNIVERSIDAD NACIONAL DE ...,CAÑARI,ROMEL DAVID,32224811,BACHILLER EN CIENCIAS DE LA EDUCACION,22/08/1997,-,Sin información (***),Sin información (***) UNIVERSIDAD NACIONAL DE ...


In [42]:
elementos

{'Apellido': ['CAÑARI'],
 'Nombre': ['ROMEL DAVID'],
 'DNI': ['32224811'],
 'Grado': ['BACHILLER EN CIENCIAS DE LA EDUCACION'],
 'Fecha Diploma': ['22/08/1997'],
 'Modalidad': ['-'],
 'Fecha Matrícula': ['Sin información (***)'],
 'Fecha Egreso': ['Sin información (***) UNIVERSIDAD NACIONAL DE EDUCACIÓN ENRIQUE GUZMÁN Y VALLE']}

In [43]:
patron = r'(\w+), (\w+ [\w ]+)\nDNI (\d+)\n([^\n]+)\n\nFecha de diploma: (\d{1,2}/\d{1,2}/\d{4})\nModalidad de estudios: ([^\n]+)\n\nFecha matrícula: ([^\n]+)\nFecha egreso: ([^\n]+) ([^\n]+)\n((?:[^\n]+\n)+)([^\n]+)'
columnas = ['Apellido', 'Nombre', 'DNI', 'Grado', 'Fecha Diploma', 'Modalidad', 'Fecha Matrícula', 'Fecha Egreso', 'Institución']

# Crear un diccionario para almacenar las listas de valores para cada elemento
elementos = {col: [] for col in columnas}

# Iterar sobre cada fila del DataFrame
for index, row in df.iterrows():
    texto = row['texto']
    
    # Buscar coincidencias en el texto
    match = re.search(patron, texto)
    if match:
        elementos['Apellido'].append(match.group(1))
        elementos['Nombre'].append(match.group(2))
        elementos['DNI'].append(match.group(3))
        elementos['Grado'].append(match.group(4))
        elementos['Fecha Diploma'].append(match.group(5))
        elementos['Modalidad'].append(match.group(6))
        elementos['Fecha Matrícula'].append(match.group(7))
        elementos['Fecha Egreso'].append(match.group(8) + " " + match.group(9))
        # Extraer la institución de la última parte del texto
        elementos['Institución'].append(match.group(10).strip())

# Crear un nuevo DataFrame con las columnas específicas
df_resultado = pd.DataFrame(elementos)

# Concatenar el nuevo DataFrame con el original
df = pd.concat([df, df_resultado], axis=1)

In [46]:
df.to_excel( 'hola.xlsx' )

In [26]:
grado = driver.find_element( By.XPATH, '//*[@id="finalData"]/tr/td[2]/b[1]/p' ).text
grado

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="finalData"]/tr/td[2]/b[1]/p"}
  (Session info: chrome=119.0.6045.124)
Stacktrace:
	GetHandleVerifier [0x009772A3+45731]
	(No symbol) [0x00902D51]
	(No symbol) [0x007F880D]
	(No symbol) [0x0082B940]
	(No symbol) [0x0082BE0B]
	(No symbol) [0x0085D1F2]
	(No symbol) [0x00848024]
	(No symbol) [0x0085B7A2]
	(No symbol) [0x00847DD6]
	(No symbol) [0x008231F6]
	(No symbol) [0x0082439D]
	GetHandleVerifier [0x00C80716+3229462]
	GetHandleVerifier [0x00CC84C8+3523784]
	GetHandleVerifier [0x00CC214C+3498316]
	GetHandleVerifier [0x00A01680+611968]
	(No symbol) [0x0090CCCC]
	(No symbol) [0x00908DF8]
	(No symbol) [0x00908F1D]
	(No symbol) [0x008FB2C7]
	BaseThreadInitThunk [0x7760FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D07C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77D07C3E+238]


In [43]:
fecha_diploma = driver.find_element( By.XPATH, '//*[@id="finalData"]/tr/td[2]/b[2]' ).text
fecha_diploma

'Fecha de diploma: 01/03/2023'

In [44]:
modalidad = driver.find_element( By.XPATH, '//*[@id="finalData"]/tr/td[2]/span[1]' ).text
modalidad

'PRESENCIAL'

In [45]:
institucion = driver.find_element( By.XPATH, '//*[@id="finalData"]/tr/td[3]/span' ).text
institucion

'PONTIFICIA UNIVERSIDAD CATÓLICA DEL PERÚ'

In [49]:
def extract_info( df ):
    
    options = Options()
    service = Service( ChromeDriverManager().install( ) )
    driver  = webdriver.Chrome( service = service )
    driver.maximize_window()

    url     = f'https://enlinea.sunedu.gob.pe/'
    driver.get( url )
    wait    = WebDriverWait( driver, 60 )
    
    verificar_titulos_popup = driver.find_element( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' )
    verificar_titulos_popup.click()
    
    frame_popup = driver.find_element( By.XPATH, '//*[@id="ifrmShowFormConstancias"]' )
    driver.switch_to.frame( frame_popup )
    
    for index, row in df.iterrows():
        
        dni_valor = row['dni']
        
        dni_campo = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="doc"]' ) ) )
        dni_campo.clear()
        dni_campo.send_keys( dni_valor )
                
        captcha       = driver.find_element( By.XPATH, '//*[@id="captchaImg"]/img' )
        captcha_image = captcha.screenshot_as_png
        with open( 'image.png', 'wb' ) as f:
            f.write( captcha_image )

        clave_valor = pytesseract.image_to_string( Image.open( 'image.png') ).strip()
        campo_captcha = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="captcha"]' ) ) )
        campo_captcha.clear()
        campo_captcha.send_keys( clave_valor )                
            
        buscar_boton = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="buscar"]' ) ) )
        buscar_boton.click()
        
        grado         = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[1]/p' ) ) ).text
        fecha_diploma = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[2]' ) ) ).text
        modalidad     = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/span' ) ) ).text
        institucion   = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[3]/span' ) ) ).text
        
        print( grado )
        print( institucion )
        
        df.at[ index, 'grado' ]         = grado       
        df.at[ index, 'fecha_diploma' ] = fecha_diploma    
        df.at[ index, 'modalidad' ]     = modalidad    
        df.at[ index, 'institucion' ]   = institucion
        
        # Salir del iframe para recargar la página principal
        driver.switch_to.default_content()

        # Recargar la página principal
        driver.refresh()

        # Volver a entrar en el iframe
        verificar_titulos_popup = driver.find_element( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' )
        verificar_titulos_popup.click()
        frame_popup = driver.find_element(By.XPATH, '//*[@id="ifrmShowFormConstancias"]')
        driver.switch_to.frame(frame_popup)

        time.sleep(10)
        
    driver.quit()
        
    return df

In [50]:
data = pd.read_excel( 'data.xlsx', dtype = { 'dni': str } )
data

,dni
0,70313538
1,32220818
2,32224811


In [51]:
new_data = extract_info( data )

Bachiller en Ciencias Sociales con Mención en Ciencia Política y Gobierno
PONTIFICIA UNIVERSIDAD CATÓLICA DEL PERÚ
LICENCIADA EN EDUCACION PRIMARIA
UNIVERSIDAD PRIVADA DE SAN PEDRO
LICENCIADO EN CIENCIAS DE LA EDUCACION
UNIVERSIDAD NACIONAL DE EDUCACIÓN ENRIQUE GUZMÁN Y VALLE


In [48]:
new_data

,dni,grado,fecha_diploma,modalidad,institucion
0,70313538,Bachiller en Ciencias Sociales con Mención en ...,Fecha de diploma: 01/03/2023,PRESENCIAL,PONTIFICIA UNIVERSIDAD CATÓLICA DEL PERÚ
1,32220818,LICENCIADA EN EDUCACION PRIMARIA,Fecha de diploma: 01/06/2007,-,UNIVERSIDAD PRIVADA DE SAN PEDRO
2,32224811,LICENCIADO EN CIENCIAS DE LA EDUCACION,Fecha de diploma:,-,UNIVERSIDAD NACIONAL DE EDUCACIÓN ENRIQUE GUZM...


In [ ]:
//*[@id="finalData"]/tr/td[2]/span[1]
//*[@id="finalData"]/tr[2]/td[2]/span

In [ ]:
//*[@id="finalData"]/tr[1]/td[2]/span

In [ ]:
## SI FUNCIONA

def extract_info( df ):
    
    options = Options()
    service = Service( ChromeDriverManager().install( ) )
    driver  = webdriver.Chrome( service = service )
    driver.maximize_window()

    url     = f'https://enlinea.sunedu.gob.pe/'
    driver.get( url )
    wait    = WebDriverWait( driver, 60 )
    
    verificar_titulos_popup = driver.find_element( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' )
    verificar_titulos_popup.click()
    
    frame_popup = driver.find_element( By.XPATH, '//*[@id="ifrmShowFormConstancias"]' )
    driver.switch_to.frame( frame_popup )
    
    for index, row in df.iterrows():
        
        dni_valor = row['dni']
        
        dni_campo = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="doc"]' ) ) )
        dni_campo.clear()
        dni_campo.send_keys( dni_valor )
        

        max_retries = 5

        for retry in range( max_retries ):
            
            try:
                
                captcha       = driver.find_element( By.XPATH, '//*[@id="captchaImg"]/img' )
                captcha_image = captcha.screenshot_as_png
                with open( 'image.png', 'wb' ) as f:
                    f.write( captcha_image )
                
                clave_valor = pytesseract.image_to_string( Image.open( 'image.png') ).strip()
                campo_captcha = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="captcha"]' ) ) )
                campo_captcha.clear()
                campo_captcha.send_keys( clave_valor )                 
                break  # Salir del bucle si el bloque try tiene éxito
            
            except:
                print( f'Error: { e }' )
                print( f'Intentando de nuevo después de 30 segundos...' )
                time.sleep( 30 )
                boton_x = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="closeModalError"]/span[1]' ) ) )
                boton_x.click()
        
        buscar_boton = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="buscar"]' ) ) )
        buscar_boton.click()
        
        grado         = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[1]/p' ) ) ).text
        fecha_diploma = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[2]' ) ) ).text
        modalidad     = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/span' ) ) ).text
        institucion   = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[3]/span' ) ) ).text
        
        print( grado )
        print( institucion )
        
        df.at[ index, 'grado' ]         = grado       
        df.at[ index, 'fecha_diploma' ] = fecha_diploma    
        df.at[ index, 'modalidad' ]     = modalidad    
        df.at[ index, 'institucion' ]   = institucion
        
        # Salir del iframe para recargar la página principal
        driver.switch_to.default_content()

        # Recargar la página principal
        driver.refresh()

        # Volver a entrar en el iframe
        verificar_titulos_popup = driver.find_element( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' )
        verificar_titulos_popup.click()
        frame_popup = driver.find_element(By.XPATH, '//*[@id="ifrmShowFormConstancias"]')
        driver.switch_to.frame(frame_popup)

        time.sleep(10)
        
    driver.quit()
        
    return df